In [16]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [17]:
from graphdatascience import GraphDataScience 

gds = GraphDataScience( 
    os.environ[ "NEO4J_URI" ], 
    auth=(os.environ[ "NEO4J_USERNAME" ], os.environ[ "NEO4J_PASSWORD" ]) 
)

In [18]:
gds.graph.drop("communities")
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [19]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

Component count: 2
Component distribution: {'min': 4, 'p5': 4, 'max': 26, 'p999': 26, 'p99': 26, 'p1': 4, 'p10': 4, 'p90': 26, 'p50': 4, 'p25': 4, 'p75': 26, 'p95': 26, 'mean': 15.0}


In [20]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

writeMillis                                                             22
nodePropertiesWritten                                                   30
ranLevels                                                                2
didConverge                                                           True
nodeCount                                                               30
communityCount                                                           5
communityDistribution    {'min': 3, 'p5': 3, 'max': 12, 'p999': 12, 'p9...
modularity                                                        0.543721
modularities                     [0.48060486522024987, 0.5437212360289283]
postProcessingMillis                                                     7
preProcessingMillis                                                      0
computeMillis                                                           30
configuration            {'writeProperty': 'communities', 'theta': 0.01...
Name: 0, dtype: object

In [21]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

[{'count(*)': 60}]

In [22]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:HAS_ENTITY]-(d:__Chunk__)
WITH c, count(distinct d) AS rank
SET c.rank = rank;
""")

[]

In [23]:
import pandas as pd
import numpy as np

community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

,Level,Number of communities,25th Percentile,50th Percentile,75th Percentile,90th Percentile,99th Percentile,Max
0,0,10,2.0,3.0,3.75,4.1,4.91,5
1,1,5,4.0,5.0,6.00,9.6,11.76,12


In [24]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WHERE c.level IN [0,1,4]
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | {id: n.id, description: n.description, type: [el in labels(n) WHERE el <> '__Entity__'][0]}] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")
community_info

[{'communityId': '0-3',
  'nodes': [{'id': 'Company', 'description': None, 'type': 'Organization'},
   {'id': 'Policyholder', 'description': None, 'type': 'Person'},
   {'id': 'Value-Added Feedback Bonus',
    'description': None,
    'type': 'Concept'},
   {'id': 'Insured', 'description': None, 'type': 'Person'}],
  'rels': [{'start': 'Company',
    'description': None,
    'type': 'REFUND_ACCUMULATED_AMOUNT',
    'end': 'Insured'},
   {'start': 'Company',
    'description': None,
    'type': 'NOTIFY_IN_WRITING',
    'end': 'Policyholder'},
   {'start': 'Company',
    'description': None,
    'type': 'REFUND_ACCUMULATED_AMOUNT',
    'end': 'Policyholder'},
   {'start': 'Value-Added Feedback Bonus',
    'description': None,
    'type': 'PURCHASE_ADDITIONAL_PAID-UP_INSURANCE',
    'end': 'Policyholder'},
   {'start': 'Value-Added Feedback Bonus',
    'description': None,
    'type': 'DEDUCT_PREMIUM',
    'end': 'Policyholder'}]},
 {'communityId': '0-5',
  'nodes': [{'id': '投保', 'descrip

In [25]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

community_template = """請根據提供同社區的資訊包含 nodes 與 relationships,
產生自然語言的摘要 同社區的資訊:
{community_info}

Summary:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input triples, generate the information summary. No pre-amble.",
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [27]:
def prepare_string(data):
    nodes_str = "Nodes are:\n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}\n"

    rels_str = "Relationships are:\n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}\n"

    return nodes_str + "\n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

process_community(community_info[0])

{'community': '0-3',
 'summary': '在這個社區中，有幾個重要的節點和關係。節點包括一個公司 (Company)，兩個人 (Policyholder 和 Insured)，以及一個概念 (Value-Added Feedback Bonus)。公司與被保險人 (Insured) 和保單持有人 (Policyholder) 之間存在累積退款的關係。此外，公司會以書面形式通知保單持有人。增值反饋獎金 (Value-Added Feedback Bonus) 則與購買額外付費保險和扣除保費相關，這些操作都是針對保單持有人的。'}

In [28]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

summaries = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_community, community): community for community in community_info}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
        summaries.append(future.result())

Processing communities: 100%|██████████| 15/15 [00:06<00:00,  2.35it/s]


In [29]:
summaries

[{'community': '0-27', 'summary': '從被保險人滿15足歲之日起，約定發生效力於死亡給付。'},
 {'community': '0-5',
  'summary': '台灣人壽是一家提供保險產品的組織，其中包括美鑫美利美元利率變動型終身壽險。人們可以投保這種保險產品。'},
 {'community': '0-21', 'summary': '本公司是一個組織，它與身故保險金這一概念存在關聯。本公司負責給付身故保險金。'},
 {'community': '0-23',
  'summary': '在這個社區中，有四個節點和三個關係。節點包括概念「書面通知」、「保險金額」、「電子郵件通知」，以及一個人「要保人」。關係方面，「書面通知」和「電子郵件通知」都通知了「要保人」。「要保人」則選擇了「保險金額」。'},
 {'community': '0-10',
  'summary': '被保險人是一個人，其中包含一個保險年齡的屬性。這兩個節點之間的關係是，被保險人具有一個特定的保險年齡。'},
 {'community': '0-29',
  'summary': '在這個社區中，有一個概念節點 "保險契約" 和一個日期節點 "民國九十九年二月三日"。這兩個節點之間存在一個關係，表示 "民國九十九年二月三日" 是 "保險契約" 訂立的日期。'},
 {'community': '0-18',
  'summary': '在這個社區中，包含了一些重要的節點和關係：\n\n1. 節點:\n- "本契約終止" (事件)\n- "受益人" (人員)\n- "保險金" (概念)\n\n2. 關係:\n- "本契約終止" 事件發生後，"受益人" 會受領相關事宜。\n- "受益人" 會受領 "保險金"。\n\n總結來說，當 "本契約終止" 時，"受益人" 將會受領到 "保險金"。'},
 {'community': '0-12',
  'summary': '在這個社區中，我們有五個節點，分別是「儲存生息」、「保單年度」、「各年度之增值回饋分享金」、「宣告利率」和「年複利方式」。\n\n「儲存生息」會影響「各年度之增值回饋分享金」。「各年度之增值回饋分享金」與「保單年度」相關，並且在「保單年度」開始給付，同時這些回饋金會累積到「年複利方式」中。此外，

In [30]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

[]